In [1]:
from engineerPMF import *
import seaborn as sns
import pandas as pd
from ipywidgets import Layout, interact, IntSlider, FloatSlider, Dropdown

### Select custom-poled KTP configuration from data

In [2]:
def chosePoling(poling):
    # Get poling configuration from file or periodically poled KTP
    # poling is a Boolean variable
    
    filename = "./data/polingConfig_032.txt" # best config from stimulated annealing
    
    if poling == "Custom-poled":
        polingConfig = np.concatenate((readPolingConfigFile(filename), np.ones((7,))))
        NUMDOMAIN = len(polingConfig)         
    elif poling == "Periodically-poled": 
        NUMDOMAIN = 650
        polingConfig = [(-1)**k for k in range(NUMDOMAIN)]
    return NUMDOMAIN, polingConfig

## Pump laser setting

In [3]:
FWHM = 0.63E-9 # FWHM for Guassian pump (0.63nm for 1.0ps, 0.79nm for 0.8ps)
delta = 10E-9 #METER

## Initialise focusing parameter

In [4]:
pumpWaist = 300E-6 #METER
signalWaist = idlerWaist = 115E-6 #METER

## Initial joint spectrum

In [5]:
def slider(temp, domain_width, pump, poling):
    
    NUMDOMAIN, polingConfig = chosePoling(poling)
    
    crystalTemp = 12 # degree Celsius

    lo = 23.10E-6 #METERS
    lc = crystalExpansion(lo, crystalTemp, enable = True) # Set enable to False to turn of crystal expansion

    LENGTH = NUMDOMAIN*lc #MICRON
    
    pumpWavelength = pump*1E-9
    numGrid = 50+1
    
    centralSignalWavelength = 2*pumpWavelength #METER
    centralIdlerWavelength =  centralSignalWavelength # momentum conserved
    
    centralWavelength = np.array([pumpWavelength, centralSignalWavelength, centralIdlerWavelength])

    signalWavelength = np.linspace(centralSignalWavelength-delta,
                                   centralSignalWavelength+delta, numGrid)
    idlerWavelength = np.linspace(centralIdlerWavelength-delta,
                                  centralIdlerWavelength+delta, numGrid)

    # There is a need to reverse the idlerWavelength array, or else the origin will
    # start on the top left corner of grid instead of bottom left.
    
    idlerWavelength = idlerWavelength[::-1]
    signal, idler = np.meshgrid(signalWavelength, idlerWavelength)
    
    PEF = pumpEnvFunc(signal, idler, centralWavelength, FWHM, shape = 'sech')

    customPhi, axis = customPhaseMatchedFunc(domain_width*1E-6, polingConfig, temp,
                                            signal, idler, centralWavelength,
                                            pumpWaist, signalWaist, idlerWaist, segments = 8)

    JSA = customPhi*PEF
    JSI = np.abs(JSA)**2
    purity, entropy = getPurity(JSA) # JSI or JSA
    
    signalJSI = np.sum(JSI, axis=0)
    idlerJSI = np.sum(JSI, axis=1)
    
    guessSignal = (np.max(signalJSI), 2*pumpWavelength, FWHM)
    guessIdler = (np.max(idlerJSI), 2*pumpWavelength, FWHM)
    poptSignal, pcov = curve_fit(spectrafit, signalWavelength, signalJSI, guessSignal)
    poptIdler,  pcov = curve_fit(spectrafit, idlerWavelength, signalJSI, guessIdler)                          
    
    newSignal = np.linspace(centralSignalWavelength-delta,
                            centralSignalWavelength+delta, numGrid*5)
    newIdler  = np.linspace(centralIdlerWavelength-delta,
                            centralIdlerWavelength+delta, numGrid*5)[::-1]
    
    signalFit = spectrafit(newSignal, *poptSignal)
    idlerFit = spectrafit(newIdler, *poptIdler)
    
    fig = plt.figure(figsize=(18, 9))
    # Add a gridspec with two rows and two columns and a ratio of 2 to 7 between
    # the size of the marginal axes and the main axes in both directions.
    # Also adjust the subplot parameters for a square plot.
    gs = fig.add_gridspec(3, 3,  width_ratios=(8, 2, 8), height_ratios=(2, 4, 4),
                          left=0.1, right=0.9, bottom=0.1, top=0.9,
                          wspace=0.05, hspace=0.05)
    
    ax = fig.add_subplot(gs[1:, 0])
    plt.contourf(signalWavelength*1E9, idlerWavelength*1E9, JSI, levels = 100)
    plt.locator_params(axis='y', nbins=7)
    plt.locator_params(axis='x', nbins=7)
    
    sig = fig.add_subplot(gs[0, 0])
    plt.axvline(x=1550, linestyle='--', color='xkcd:dusty purple')
    plt.yticks([])
    plt.xticks([])
    plt.plot(newSignal*1E9, signalFit, linewidth=2.5, color='xkcd:avocado')
    signalCenter = "Signal central wavelength: {:0.02f} nm, signal bandwith: {:0.02f} nm".format(poptSignal[1]*1E9, poptSignal[-1]*1E9)
    
    idl = fig.add_subplot(gs[1:, 1])
    plt.axhline(y=1550, linestyle='--', color='xkcd:dusty purple')
    plt.yticks([])
    plt.xticks([])
    plt.plot(idlerFit, newIdler*1E9, linewidth=2.5, color='xkcd:terra cotta')
    idlerCenter = "Idler central wavelength: {:0.02f} nm, idler bandwith: {:0.02f} nm".format(poptIdler[1]*1E9, poptIdler[-1]*1E9)
    
    ovr = fig.add_subplot(gs[1,2])
    plt.yticks([])
    plt.xlabel("Wavelength (nm)")
    plt.locator_params(axis='y', nbins=7)
    plt.plot(newIdler*1E9, idlerFit, linewidth=2.5, color='xkcd:terra cotta')
    plt.plot(newSignal*1E9, signalFit, linewidth=2.5, color='xkcd:avocado')
    plt.axvline(x=poptSignal[1]*1E9, linestyle='--', color='xkcd:dusty purple')
    plt.axvline(x=poptIdler[1]*1E9, linestyle='--', color='xkcd:dusty purple')
    
    
    print(signalCenter)
    print(idlerCenter)
    print("The PMF axis is at: {:0.02f}".format(axis))
    print("Purity: {:0.02f}, Entropy: {:0.02f}".format(purity*1E2, entropy))
    
    return ax

# avocado
# terra cotta
# dusty purple
# light maroon
# reddish
# dark lilac

## Joint spectrum amplitude plot

In [6]:
# https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html

interact(slider,
         poling=Dropdown(value = "Custom-poled",
                         options = ["Periodically-poled", "Custom-poled"], description = "poling config"),
         temp=FloatSlider(value=32, description='temperature (Celsius)',
                          min=-100, max=100, step=0.01, continuous_update = False,
                          layout=Layout(width='60%')),
         domain_width=FloatSlider(value=23.10, description='domain width (micron)',
                                  min=22, max=24, step=0.01, continuous_update = False,
                                  layout=Layout(width='60%')),
         pump=FloatSlider(value=775, description='pump wavelength',
                          min=750, max=850, step=0.1, continuous_update = False,
                          layout=Layout(width='60%')))

interactive(children=(FloatSlider(value=32.0, continuous_update=False, description='temperature (Celsius)', la…

<function __main__.slider(temp, domain_width, pump, poling)>